## Lab 2. Object tracking

what this sample explains.

* import existing app
* how to implement object tracking layer on top of object detection
* deploy programmatically

## Preparation

In [ ]:
# Import libraries

import sys
import os
import time
import json
import glob
import tarfile

import boto3
import sagemaker
import IPython
import gluoncv

sys.path.insert( 0, os.path.abspath( "../common/test_utility" ) )
import panorama_test_utility

In [ ]:
# Initialize variables and configurations

boto3_session = boto3.session.Session()
sm_session = sagemaker.Session()

account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3_session.region_name
s3_bucket = sm_session.default_bucket()
sm_role = sagemaker.get_execution_role()

print( "account_id :", account_id )
print( "region :", region )
print( "s3_bucket :", s3_bucket )
print( "sm_role :", sm_role )

In [ ]:
# FIXME : revert account IDs in project files to placeholder

%run ./internal/revert.py

## Start with "People detection" application

In [ ]:
app_name = "lab2"

!cd {app_name} && panorama-cli import-application

In [ ]:
code_package_name = f"{app_name}_code"
code_package_version = "1.0"
source_filename = f"./lab2/packages/{account_id}-{code_package_name}-{code_package_version}/src/app.py"

In [ ]:
def export_model_and_create_targz( prefix, name, model ):
    os.makedirs( prefix, exist_ok=True )
    gluoncv.utils.export_block( os.path.join( prefix, name ), model, preprocess=False, layout="CHW" )

    tar_gz_filename = f"{prefix}/{name}.tar.gz"
    with tarfile.open( tar_gz_filename, "w:gz" ) as tgz:
        tgz.add( f"{prefix}/{name}-symbol.json", f"{name}-symbol.json" )
        tgz.add( f"{prefix}/{name}-0000.params", f"{name}-0000.params" )
        
    print( f"Exported : {tar_gz_filename}" )
    
# Export object detection model. Reset the classes for human detection only.
people_detection_model = gluoncv.model_zoo.get_model('yolo3_mobilenet1.0_coco', pretrained=True)
people_detection_model.reset_class(["person"], reuse_weights=['person'])
export_model_and_create_targz( "models", "yolo3_mobilenet1.0_coco_person", people_detection_model )

In [ ]:
model_package_name = f"{app_name}_model"
model_package_version = "1.0"
people_detection_model_name = "people_detection_model"

!cd {app_name} && panorama-cli add-raw-model \
    --model-asset-name {people_detection_model_name} \
    --model-local-path ../models/yolo3_mobilenet1.0_coco_person.tar.gz \
    --descriptor-path packages/{account_id}-{model_package_name}-{model_package_version}/descriptor.json \
    --packages-path packages/{account_id}-{model_package_name}-{model_package_version}

In [ ]:
people_detection_model_data_shape = '{"data":[1,3,480,600]}'

%run ../common/test_utility/panorama_test_utility_compile.py \
\
--s3-model-location s3://{s3_bucket}/panorama-workshop/{app_name} \
\
--model-node-name {people_detection_model_name} \
--model-file-basename ./models/yolo3_mobilenet1.0_coco_person \
--model-data-shape '{people_detection_model_data_shape}' \
--model-framework MXNET

In [ ]:
video_filepath = "../../videos/TownCentreXVID.avi"

%run ../common/test_utility/panorama_test_utility_run.py \
\
--app-name {app_name} \
--code-package-name {code_package_name} \
--py-file {source_filename} \
\
--model-package-name {model_package_name} \
--model-node-name {people_detection_model_name} \
--model-file-basename ./models/yolo3_mobilenet1.0_coco_person \
\
--camera-node-name lab2_camera \
\
--video-file {video_filepath} \
--video-start 0 \
--video-stop 10 \
--video-step 1 \
\
--output-screenshots ./screenshots/%Y%m%d_%H%M%S

In [ ]:
# View latest screenshot image

latest_screenshot_dirname = sorted( glob.glob( "./screenshots/*" ) )[-1]
screenshot_filename = sorted( glob.glob( f"{latest_screenshot_dirname}/*.png" ) )[-1]

print(screenshot_filename)
IPython.display.Image( filename = screenshot_filename )

## Extend to "People tracking" application

<div class="alert alert-block alert-warning"><b>Manual edit needed:</b> Manually edit the app.py referring to README.md file.</div>

In [ ]:
video_filepath = "../../videos/TownCentreXVID.avi"

%run ../common/test_utility/panorama_test_utility_run.py \
\
--app-name {app_name} \
--code-package-name {code_package_name} \
--py-file {source_filename} \
\
--model-package-name {model_package_name} \
--model-node-name {people_detection_model_name} \
--model-file-basename ./models/yolo3_mobilenet1.0_coco_person \
\
--camera-node-name lab2_camera \
\
--video-file {video_filepath} \
--video-start 0 \
--video-stop 10 \
--video-step 1 \
\
--output-screenshots ./screenshots/%Y%m%d_%H%M%S

In [ ]:
# View latest screenshot image

latest_screenshot_dirname = sorted( glob.glob( "./screenshots/*" ) )[-1]
screenshot_filename = sorted( glob.glob( f"{latest_screenshot_dirname}/*.png" ) )[-1]

print(screenshot_filename)
IPython.display.Image( filename = screenshot_filename )

## Run the people tracking application on real device

In [ ]:
panorama_client = boto3.client("panorama")

In [ ]:
response = panorama_client.list_devices()
for device in response["Devices"]:
    if device["ProvisioningStatus"]=="SUCCEEDED":
        break
else:
    assert False, "Provisioned device not found."

device_id = device["DeviceId"]

print( "%s : %s" % (device["Name"], device["DeviceId"]) )

<div class="alert alert-block alert-warning"><b>Manual operation needed:</b> Look up data source name on the Management Console, referring to README.md file.</div>

In [ ]:
!cd {app_name} && panorama-cli build-container --container-asset-name code --package-path packages/{account_id}-{code_package_name}-{code_package_version}

In [ ]:
!cd {app_name} && panorama-cli package-application

<div class="alert alert-block alert-warning"><b>Manual edit needed:</b> Manually edit override manifest file referring to README.md file.</div>

Template of override manifest file

``` json
{
    "nodeGraphOverrides": {
        "envelopeVersion": "2021-01-01",
        "packages": [
            {
                "name": "{YourAwsAccountId}::{YourCameraName}",
                "version": "1.0"
            }
        ],
        "nodes": [
            {
                "name": "{YourCameraName}_node",
                "interface": "{YourAwsAccountId}::{YourCameraName}.{YourCameraName}",
                "overridable": true,
                "overrideMandatory" : false,
                "launch" : "onAppStart"
            }
        ],
        "nodeOverrides": [
            {
                "replace": "lab2_camera",
                "with": [
                    {
                        "name": "{YourCameraName}_node"
                    }
                ]
            }
        ]
    }
}
```


In [ ]:
def get_escaped_payload_from_json(filename):
    with open(filename) as fd:
        return json.dumps(json.loads(fd.read()))

manifest_payload = get_escaped_payload_from_json( f"./{app_name}/graphs/{app_name}/graph.json" )
override_payload = get_escaped_payload_from_json( f"./{app_name}/graphs/{app_name}/override.json" )

response = panorama_client.create_application_instance(
    Name = app_name,
    DefaultRuntimeContextDevice = device_id,
    ManifestPayload = {"PayloadData":manifest_payload},
    ManifestOverridesPayload = {"PayloadData":override_payload},
)

application_instance_id = response["ApplicationInstanceId"]

response


In [ ]:
def wait_deployment( application_instance_id ):
    
    progress_dots = panorama_test_utility.ProgressDots()    
    while True:
        app = panorama_client.describe_application_instance( ApplicationInstanceId = application_instance_id )
        progress_dots.update_status( "%s (%s)" % (app["Status"], app["StatusDescription"]) )
        if app["Status"] not in ( "DEPLOYMENT_PENDING", "DEPLOYMENT_REQUESTED", "DEPLOYMENT_IN_PROGRESS" ):
            break
        time.sleep(60)

wait_deployment( application_instance_id )

In [ ]:
logs_url = panorama_test_utility.get_logs_url( region, device_id, application_instance_id )
print( "CloudWatch Logs URL :" )
print( logs_url )

<div class="alert alert-block alert-warning"><b>Manual operation needed:</b> Confirm the application is running as expected by visitting the CloudWatch Logs link above and seeing HDMI display.</div>

In [ ]:
panorama_test_utility.remove_application( device_id, application_instance_id )